In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from dolfin import *

# Sub domain for Dirichlet boundary condition
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (abs(x[0]) < DOLFIN_EPS) or (abs(x[0] - 1.0) < DOLFIN_EPS) 
        #return abs(x[0] - 1.0) < DOLFIN_EPS and on_boundary

# Create mesh and define function space
mesh = UnitSquareMesh(100, 100)
File("mesh.pvd") << mesh

V = FunctionSpace(mesh, "CG", 1)

# The second argument to :py:class:`FunctionSpace
# <dolfin.cpp.function.FunctionSpace>` is the finite element family,
# while the third argument specifies the polynomial degree. Thus, in
# this case, we use 'CG', for Continuous Galerkin, as a synonym for
# 'Lagrange'. With degree 1, we simply get the standard linear Lagrange
# element, which is a triangle with nodes at the three vertices (or in
# other words, continuous piecewise linear polynomials).
# 
# The Dirichlet boundary condition can be created using the class
# :py:class:`DirichletBC <dolfin.cpp.fem.DirichletBC>`. A
# :py:class:`DirichletBC <dolfin.cpp.fem.DirichletBC>` takes three
# arguments: the function space the boundary condition applies to, the
# value of the boundary condition, and the part of the boundary on which
# the condition applies. In our example, the function space is V, the
# value of the boundary condition (1.0) can be represented using a
# Constant and the Dirichlet boundary is defined above. The definition
# of the Dirichlet boundary condition then looks as follows::

# Define boundary condition
g = Constant(313.0)

# Define variational problem
u = Function(V)
v = TestFunction(V)

u_0 = 4000
u = project(u_0,V)

sigma = 1.387 * pow(10,-23)
epsilon = 0.73
T_0 = 313.0
h_conv = 60.0
# kappa_0 = 2.03

kappa_0 = Expression("((x[0]>0.35) && (x[0]<0.65) && (x[1]>0.79) && (x[1]<0.99))?0.8:2.03", degree=1)
rho_0 = Expression("((x[0]>0.35) && (x[0]<0.65) && (x[1]>0.79) && (x[1]<0.99))?2500:800", degree=1)
c_0 = Expression("((x[0]>0.35) && (x[0]<0.65) && (x[1]>0.79) && (x[1]<0.99))?840:830", degree=1)

g2 = Expression("((abs(x[1] - 1.0) < 1e-4) && (x[0]>0.49) && (x[0]<0.51)) ? 600000.0:0.0", degree=1)
F = inner(grad(u), grad(v))*dx - kappa_0*g2*v*ds + kappa_0*epsilon*sigma*u**4*v*ds - kappa_0*epsilon*sigma*T_0**4*v*ds + kappa_0*h_conv*(u)*v*ds - kappa_0*h_conv*(T_0)*v*ds

# Compute solution
solve(F == 0, u, solver_parameters={"newton_solver":{"relative_tolerance":1e-3},
                                   "newton_solver":{"maximum_iterations":1000}})

# Plot solution and solution gradient
plt.figure()
p = plot(u)

plt.colorbar(p)
plt.interactive(True)

plt.figure()
plot(grad(u), title="Solution gradient")

plt.show()

# Save solution in VTK format
file = File("nonlinear_poisson.pvd")
file << u